# Compiling Go Source Code

In [1]:
code = r'''
package main

import (
    "chain"
)

//contract hello
type MyContract struct {
    Receiver      chain.Name
    FirstReceiver chain.Name
    Action        chain.Name
}

func NewContract(receiver, firstReceiver, action chain.Name) *MyContract {
    return &MyContract{receiver, firstReceiver, action}
}

//action sayhello
func (c *MyContract) SayHello(name string) {
    println("Hello", name)
}
'''
with open('test.go', 'w') as f:
    f.write(code)

In [2]:
%system eosio-go -o test.wasm test.go

['2021/08/26 13:50:45 parse file: test.go',
 '2021/08/26 13:50:45 ++struct: MyContract',
 '2021/08/26 13:50:45 +++action sayhello',
 'wasm-ld --allow-undefined --stack-first --export-dynamic --no-demangle -zstack-size=8192 -L /Users/newworld/dev/github/tinygo -o /var/folders/pv/678xdw3s56xggsvz36r65kzr0000gp/T/tinygo277563680/main /Users/newworld/dev/github/tinygo/lib/wasi-libc-eosio/sysroot/lib/wasm32-wasi/libc.a /var/folders/pv/678xdw3s56xggsvz36r65kzr0000gp/T/tinygo277563680/main.o']

In [3]:
%system ls -l *.wasm

['-rwxr-xr-x  1 newworld  staff  131 Aug 26 13:50 test.wasm']

In [4]:
%system ls -l *.abi

['-rw-r--r--  1 newworld  staff  563 Aug 26 13:50 hello.abi']

# Testing

In [5]:
import os
from uuoskit import uuosapi, wallet, config

python_contract = config.python_contract
test_account1 = 'helloworld11'
test_account2 = 'helloworld12'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')


# active key of helloworld11
wallet.import_key('test', '5JRYimgLBrRLCBAcjHUWCYRv3asNedTYYzVgmiU4q2ZVxMBiJXL')
wallet.import_key('test', '5Jbb4wuwz8MAzTB9FJNmrVYGXo4ABb7wqPVoWGcZ6x8V2FwNeDo')
# active key of helloworld12
wallet.import_key('test', '5JHRxntHapUryUetZgWdd3cg6BrpZLMJdqhhXnMaZiiT4qdJPhv')

config.python_contract = 'hello'
config.main_token = 'EOS'
config.main_token_contract = 'eosio.token'
uuosapi.set_node('https://testnode.uuos.network:8443')

In [10]:
code = r'''
package main

import (
    "chain"
)

//contract hello
type MyContract struct {
    Receiver      chain.Name
    FirstReceiver chain.Name
    Action        chain.Name
}

func NewContract(receiver, firstReceiver, action chain.Name) *MyContract {
    return &MyContract{receiver, firstReceiver, action}
}

//action sayhello
func (c *MyContract) SayHello(name string) {
    println("Hello", name)
}
'''
code, abi = uuosapi.compile('hello', code, src_type=2)
print('done!')

done!


In [11]:
try:
    uuosapi.deploy_contract(test_account1, code, abi, vm_type=0)
except Exception as e:
    print(e.json['error']['what'])
print('done!')

done!


In [12]:
r = uuosapi.push_action(test_account1, 'sayhello', {'name': 'alice'}, {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

Hello alice

512
